In [2]:
!pip install peft
! pip install flask-ngrok
! pip install pyngrok
import random
import torch
from transformers import (BertForTokenClassification, AutoTokenizer,BertForMaskedLM)
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from peft import LoraModel, LoraConfig, get_peft_model
import torch
import nltk
import torch.nn.functional as F
from nltk.tokenize import word_tokenize
from peft import PeftConfig, PeftModel
import copy
from flask import Flask, send_from_directory,jsonify,request
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

BERT = BertForTokenClassification.from_pretrained('google-bert/bert-base-cased',num_labels = 2)
Tokenizer =  AutoTokenizer.from_pretrained('google-bert/bert-base-cased')

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

## Get the index of the incorrect words using the trained BERT token classifier

In [3]:
def get_incorrect_word(model , sentence):
    tokenized_text = word_tokenize(sentence)
    #print(tokenized_text)
    tokenized_inputs = Tokenizer(tokenized_text, truncation=True ,is_split_into_words=True)
    
    input_ids = torch.tensor(tokenized_inputs['input_ids']).unsqueeze(0)
    
    prediction = model(input_ids).logits.argmax(2)[0]
    
    #print(prediction)
    
    word_ids = tokenized_inputs.word_ids()
    
    previous_word_idx = None
    
    true_predictions = []
    
    for i,word_idx in enumerate(word_ids):
        #print(word_idx)
        if word_idx != None and word_idx != previous_word_idx:
            true_predictions.append(prediction[i].item())
        previous_word_idx = word_idx
        
    true_predictions = torch.tensor(true_predictions)
    
    indices = torch.nonzero(true_predictions == 1, as_tuple=False).squeeze()
    #print(indices)

    if indices.numel() == 0:
        return tokenized_text, []
        
    if indices.dim() == 0:  
        indices = indices.unsqueeze(0)
    
    incorrect_words = [tokenized_text[i.item()] for i in indices]
    incorrect_word_indices = [i.item() for i in indices]
        
    return tokenized_text, incorrect_word_indices

## Get the candidate correction words using BERT Masked Language model

In [347]:
def get_corrections(LM_model, tokenized_text, incorrect_indices):
    
    i = 0
    deleted_indices = []
    lst = copy.deepcopy(incorrect_indices)
    #print(lst)
    while i < len(lst) - 1:
        if lst[i] + 1 == lst[i + 1]:
            deleted_indices.append(lst[i + 1])
            lst.pop(i+1)
            i+=1
        else:
            i += 1
    
    #print(deleted_indices)
    t_error = copy.deepcopy(tokenized_text)
    
    #t_error = [t_error[i] for i in range(len(t_error)) if i not in deleted_indices]
    
    #print(t_error)
    # replace with [MASK] tag
    for i in lst:
        t_error[i]="[MASK]"
        
    #print(t_error)
    
    inputs = Tokenizer(t_error, return_tensors="pt",is_split_into_words=True)
    with torch.no_grad():
        logits = LM_model(**inputs).logits

    mask_token_index = (inputs.input_ids == Tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    
    probs = F.softmax(logits,dim=-1)
    top_k=10
    top_probs = torch.topk(probs[0, mask_token_index], top_k, sorted=True).indices
    #print(top_probs)
    highest_probs = top_probs[:,0]
    #print(highest_probs)
    #print(incorrect_indices)
    for idx,highest_prob_idx in zip(lst,highest_probs):
        t_error[idx] = Tokenizer.decode(highest_prob_idx.item())
        
    return " ".join(t_error)

In [5]:
Masked_LMBERT = BertForMaskedLM.from_pretrained("google-bert/bert-base-cased")
Reloaded_BERTModel = BertForTokenClassification.from_pretrained('/kaggle/input/bert-trained-spellcheck/Bert for spell check')

Some weights of the model checkpoint at google-bert/bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:

inference_model = PeftModel.from_pretrained(copy.deepcopy(Reloaded_BERTModel), "/kaggle/input/bert-trained-spellcheck/Lora Adapter/LORA")

In [343]:
texts, ids = get_incorrect_word(inference_model , """the car done hit by truck""")

In [344]:
ids

[2]

In [345]:
texts

['the', 'car', 'done', 'hit', 'by', 'truck']

In [348]:
get_corrections(Masked_LMBERT, texts, ids)

'the car was hit by truck'

## Flask backend with ngrok to host the webserver

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)
path = '/kaggle/input/bertspellcheckfrontendv3/BERT Spellchecker web app'

@app.route("/")
def html():
    return send_from_directory(path, 'index.html')

@app.route('/pexels-iriser-1590549.jpg')
def image():
    return send_from_directory(path, 'pexels-iriser-1590549.jpg')

@app.route('/style.css')
def style():
    return send_from_directory(path, 'style.css')

@app.route('/script.js')
def script():
    return send_from_directory(path, 'script.js')

data_store = []
@app.route('/submit', methods=['POST'])
def submit():
    data = request.get_json()
    text_input = data.get('txtinput', '')

    tokenized_text, indices = get_incorrect_word(inference_model , text_input)
    if len(indices)>0:
        correction = get_corrections(Masked_LMBERT, tokenized_text, indices)
        return jsonify({
        'tokenizedText': tokenized_text,
        'indices': indices,
        'correction':correction})


    return jsonify({
        'tokenizedText': tokenized_text,
        'indices': indices
    })

if __name__ == '__main__':
    ngrok.kill()

    ngrok.set_auth_token('GIVE YOUR NGROK TOKEN')

    public_url = ngrok.connect(addr='5000')
    print(f'Ngrok Tunnel URL: {public_url}')

    app.run()

Ngrok Tunnel URL: NgrokTunnel: "https://58ac-34-105-117-107.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off
Usage of ngrok requires a verified account and authtoken.

Sign up for an account: https://dashboard.ngrok.com/signup
Install your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken

ERR_NGROK_4018

 * Running on http://58ac-34-105-117-107.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
